## Model Interpretability

The following tasks are to be completed in the assignement:

1. Fit a linear model and interpret the regression coefficients

2. Fit a tree-based model and interpret the nodes

3.  Use auto ml to find the best model

4. Run SHAP analysis on the models from steps 1, 2, and 3,   interpret the SHAP values and compare them with the other model interpretability methods.

### Importing and Cleaning the Dataset


In [1]:
# Required Libraries
import pandas as pd

In [6]:
# Dataset is hosted on my github repository
url = "https://raw.githubusercontent.com/midhunmohank/DS/master/Datasets/AB_NYC_2019.csv"
df = pd.read_csv(url)

In [ ]:
import numpy as np
from sklearn import linear_model
from sklearn.model_selection import train_test_split

In [8]:
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


In [9]:
df.describe()

,id,host_id,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
count,4.889500e+04,4.889500e+04,48895.000000,48895.000000,48895.000000,48895.000000,48895.000000,38843.000000,48895.000000,48895.000000
mean,1.901714e+07,6.762001e+07,40.728949,-73.952170,152.720687,7.029962,23.274466,1.373221,7.143982,112.781327
std,1.098311e+07,7.861097e+07,0.054530,0.046157,240.154170,20.510550,44.550582,1.680442,32.952519,131.622289
min,2.539000e+03,2.438000e+03,40.499790,-74.244420,0.000000,1.000000,0.000000,0.010000,1.000000,0.000000
25%,9.471945e+06,7.822033e+06,40.690100,-73.983070,69.000000,1.000000,1.000000,0.190000,1.000000,0.000000
50%,1.967728e+07,3.079382e+07,40.723070,-73.955680,106.000000,3.000000,5.000000,0.720000,1.000000,45.000000
75%,2.915218e+07,1.074344e+08,40.763115,-73.936275,175.000000,5.000000,24.000000,2.020000,2.000000,227.000000
max,3.648724e+07,2.743213e+08,40.913060,-73.712990,10000.000000,1250.000000,629.000000,58.500000,327.000000,365.000000


In [10]:
#Dropping irrelevant columns
df.drop(['latitude','longitude','last_review','id','name','host_id','host_name'],axis= 1, inplace= True)

In [11]:
df.describe()

,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
count,48895.000000,48895.000000,48895.000000,38843.000000,48895.000000,48895.000000
mean,152.720687,7.029962,23.274466,1.373221,7.143982,112.781327
std,240.154170,20.510550,44.550582,1.680442,32.952519,131.622289
min,0.000000,1.000000,0.000000,0.010000,1.000000,0.000000
25%,69.000000,1.000000,1.000000,0.190000,1.000000,0.000000
50%,106.000000,3.000000,5.000000,0.720000,1.000000,45.000000
75%,175.000000,5.000000,24.000000,2.020000,2.000000,227.000000
max,10000.000000,1250.000000,629.000000,58.500000,327.000000,365.000000


In [12]:
df.head()

,neighbourhood_group,neighbourhood,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
0,Brooklyn,Kensington,Private room,149,1,9,0.21,6,365
1,Manhattan,Midtown,Entire home/apt,225,1,45,0.38,2,355
2,Manhattan,Harlem,Private room,150,3,0,NaN,1,365
3,Brooklyn,Clinton Hill,Entire home/apt,89,1,270,4.64,1,194
4,Manhattan,East Harlem,Entire home/apt,80,10,9,0.10,1,0


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48895 entries, 0 to 48894
Data columns (total 9 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   neighbourhood_group             48895 non-null  object 
 1   neighbourhood                   48895 non-null  object 
 2   room_type                       48895 non-null  object 
 3   price                           48895 non-null  int64  
 4   minimum_nights                  48895 non-null  int64  
 5   number_of_reviews               48895 non-null  int64  
 6   reviews_per_month               38843 non-null  float64
 7   calculated_host_listings_count  48895 non-null  int64  
 8   availability_365                48895 non-null  int64  
dtypes: float64(1), int64(5), object(3)
memory usage: 3.4+ MB


In [14]:
df.isna().sum()

neighbourhood_group                   0
neighbourhood                         0
room_type                             0
price                                 0
minimum_nights                        0
number_of_reviews                     0
reviews_per_month                 10052
calculated_host_listings_count        0
availability_365                      0
dtype: int64

In [15]:
#Enoding the categorical variables with cat.codes

df['room_type'] = df['room_type'].astype('category').cat.codes


In [16]:
df['neighbourhood_group'] = df['neighbourhood_group'].astype('category').cat.codes

In [17]:
df['neighbourhood'] = df['neighbourhood'].astype('category').cat.codes

In [18]:
df.isna().sum()

neighbourhood_group                   0
neighbourhood                         0
room_type                             0
price                                 0
minimum_nights                        0
number_of_reviews                     0
reviews_per_month                 10052
calculated_host_listings_count        0
availability_365                      0
dtype: int64

In [19]:
df.fillna(df.median(), inplace=True)

In [20]:
df.columns

Index(['neighbourhood_group', 'neighbourhood', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'reviews_per_month',
       'calculated_host_listings_count', 'availability_365'],
      dtype='object')

In [21]:
target = df["price"]

In [22]:
target.shape

(48895,)

In [23]:
features = df[['neighbourhood_group', 'neighbourhood', 'room_type',
       'minimum_nights', 'number_of_reviews', 'reviews_per_month',
       'calculated_host_listings_count', 'availability_365']]

In [24]:
features.shape

(48895, 8)

In [25]:
datasetClean = pd.concat([features,target], axis = 1, join = 'inner')

In [26]:
datasetClean.shape

(48895, 9)

In [27]:
datasetClean.isna().sum()

neighbourhood_group               0
neighbourhood                     0
room_type                         0
minimum_nights                    0
number_of_reviews                 0
reviews_per_month                 0
calculated_host_listings_count    0
availability_365                  0
price                             0
dtype: int64

In [28]:
datasetClean.shape

(48895, 9)

In [29]:
datasetClean.isna().sum()

neighbourhood_group               0
neighbourhood                     0
room_type                         0
minimum_nights                    0
number_of_reviews                 0
reviews_per_month                 0
calculated_host_listings_count    0
availability_365                  0
price                             0
dtype: int64

In [30]:
target.shape

(48895,)

In [31]:
features.shape

(48895, 8)

In [32]:
df = df.reset_index()

In [33]:
df.replace([np.inf, -np.inf], np.nan, inplace=True)

In [34]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.33, random_state=42)

In [35]:
X_train.head()

,neighbourhood_group,neighbourhood,room_type,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
35706,1,28,1,1,28,2.75,1,36
2434,2,95,1,4,77,1.81,1,21
44725,2,127,0,5,2,1.00,1,132
32341,2,201,0,4,58,4.26,1,158
9648,1,214,0,2,28,0.61,1,170


In [36]:
y_train.shape

(32759,)

In [37]:
from sklearn.linear_model import LinearRegression

In [38]:
reg = LinearRegression()
reg.fit(X_train,y_train)

#Convert categroaical variables to cat codes


LinearRegression()

In [39]:
print(reg.coef_)

[ 1.05486328e+01  1.54307734e-01 -1.09089706e+02  1.09150101e-01
 -3.05681340e-01 -1.86596492e+00  1.54723866e-02  1.77320375e-01]


In [40]:
print(reg.intercept_)

163.56676712537106


In [41]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from math import sqrt
from sklearn.metrics import r2_score

In [42]:
#Predictions



In [43]:
# #Checking accuracy of predictions
# print('--Phase-1--')
# print('MAE: %f'% mean_absolute_error(y_test, lr_pred))
# print('RMSE: %f'% np.sqrt(mean_squared_error(y_test, lr_pred)))   
# print('R2 %f' % r2_score(y_test, lr_pred))



### Linear Model

In [44]:
import statsmodels.api as sm

OLS_model = sm.OLS(y_train,X_train).fit()



In [45]:
OLS_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                  price   R-squared (uncentered):                   0.287
Model:                            OLS   Adj. R-squared (uncentered):              0.286
Method:                 Least Squares   F-statistic:                              1645.
Date:                Sun, 24 Apr 2022   Prob (F-statistic):                        0.00
Time:                        17:03:46   Log-Likelihood:                     -2.2774e+05
No. Observations:               32759   AIC:                                  4.555e+05
Df Residuals:                   32751   BIC:                                  4.556e+05
Df Model:                           8                                                  
Covariance Type:            nonrobust                                                  
==================================================================================================
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
neighbourhood_group               56.0313      1.489     37.623      0.000      53.112      58.950
neighbourhood                      0.4969      0.018     27.116      0.000       0.461       0.533
room_type                        -77.5305      2.444    -31.725      0.000     -82.320     -72.741
minimum_nights                     0.3535      0.067      5.264      0.000       0.222       0.485
number_of_reviews                 -0.2337      0.038     -6.089      0.000      -0.309      -0.159
reviews_per_month                  4.0524      1.096      3.696      0.000       1.903       6.201
calculated_host_listings_count     0.0307      0.044      0.699      0.485      -0.055       0.117
availability_365                   0.2457      0.011     22.046      0.000       0.224       0.268
==============================================================================
Omnibus:                    71048.992   Durbin-Watson:                   1.979
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        460803624.805
Skew:                          19.553   Prob(JB):                         0.00
Kurtosis:                     582.712   Cond. No.                         346.
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [46]:
#Predicting the values
lr_pred = OLS_model.predict(X_test)

### Tree based Model

In [81]:
from sklearn.tree import DecisionTreeRegressor
DTree=DecisionTreeRegressor(min_samples_leaf=.0001)
DTree.fit(X_train,y_train)

DTree_score = DTree.score(X_test, y_test)
print(DTree_score)

-0.13579095803040997


In [82]:
from sklearn.metrics import r2_score,mean_absolute_error
def model_sel(model):
    y_pred = model.predict(X_test)
    print("MODEL EVALUATION")
    print("\nr2_score of the model: ")
    print(r2_score(y_test,y_pred))

In [83]:
# Model Evaluation on DecisionTreeRegressor
print("DECISION TREE REGRESSION: ")
print("-x"*25)
model_sel(DTree)

DECISION TREE REGRESSION: 
-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x
MODEL EVALUATION

r2_score of the model: 
-0.13579095803040997


## AutoML

In [60]:
import h2o
from h2o.automl import H2OAutoML


In [61]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,14 hours 28 mins
H2O_cluster_timezone:,America/New_York
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.1.1
H2O_cluster_version_age:,10 days
H2O_cluster_name:,H2O_from_python_midhunmohan_ej6s1p
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.303 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [62]:
datasetClean.isna().sum()

neighbourhood_group               0
neighbourhood                     0
room_type                         0
minimum_nights                    0
number_of_reviews                 0
reviews_per_month                 0
calculated_host_listings_count    0
availability_365                  0
price                             0
dtype: int64

In [63]:
#Reducing the size of the dataset to 50% due to permormance issues with my hardware
dataset_train = datasetClean.sample(frac=.5)
dataset_test = datasetClean.drop(dataset_train.index)

In [76]:
dataset_test = dataset_test.sample(frac=.5)

In [64]:
dataset_train.isna().sum()

neighbourhood_group               0
neighbourhood                     0
room_type                         0
minimum_nights                    0
number_of_reviews                 0
reviews_per_month                 0
calculated_host_listings_count    0
availability_365                  0
price                             0
dtype: int64

In [65]:
features_autoML = list(features.columns)
features_autoML

['neighbourhood_group',
 'neighbourhood',
 'room_type',
 'minimum_nights',
 'number_of_reviews',
 'reviews_per_month',
 'calculated_host_listings_count',
 'availability_365']

In [66]:
target_autoML = 'price'

In [67]:
dataset_train.head()

,neighbourhood_group,neighbourhood,room_type,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,price
25534,1,28,1,2,10,0.43,2,0,54
627,2,64,0,4,0,0.72,1,0,160
8379,2,201,0,4,6,0.14,2,4,449
3044,2,64,0,5,4,0.06,1,0,199
2325,2,94,1,3,61,0.81,1,285,47


In [77]:
#Converting from pandas dataframe to Automl Dataset
datasetAutoML = h2o.H2OFrame(dataset_train)
datasetAutoMLTest = h2o.H2OFrame(dataset_test)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [69]:
datasetAutoML.head()

neighbourhood_group,neighbourhood,room_type,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,price
1,28,1,2,10,0.43,2,0,54
2,64,0,4,0,0.72,1,0,160
2,201,0,4,6,0.14,2,4,449
2,64,0,5,4,0.06,1,0,199
2,94,1,3,61,0.81,1,285,47
2,206,0,2,119,2.95,1,300,150
0,38,1,3,0,0.72,1,365,100
1,19,1,2,15,1.8,2,0,65
4,122,0,5,16,0.33,1,299,150
3,125,1,1,1,0.02,1,364,70


In [70]:
aml = H2OAutoML()
aml.train(x = features_autoML, y = target_autoML, training_frame= datasetAutoML)

AutoML progress: |
08:31:30.401: AutoML: XGBoost is not available; skipping it.

███████████████████████████████████████████████████████████████| (done) 100%
Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  GBM_3_AutoML_2_20220424_83130


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,50.0,50.0,79860.0,8.0,8.0,8.0,50.0,177.0,122.38




ModelMetricsRegression: gbm
** Reported on train data. **

MSE: 40643.80510493697
RMSE: 201.6030880342287
MAE: 63.990984779693605
RMSLE: NaN
Mean Residual Deviance: 40643.80510493697

ModelMetricsRegression: gbm
** Reported on cross-validation data. **

MSE: 50807.71688754925
RMSE: 225.40567181761256
MAE: 69.38137179833414
RMSLE: NaN
Mean Residual Deviance: 50807.71688754925

Cross-Validation Metrics Summary: 


,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,mae,69.481380,2.461434,72.708550,67.460365,66.845184,71.13541,69.257380
1,mean_residual_deviance,50880.660000,10378.033000,56318.594000,40530.234000,44589.066000,46690.82400,66274.586000
2,mse,50880.660000,10378.033000,56318.594000,40530.234000,44589.066000,46690.82400,66274.586000
3,r2,0.156957,0.034202,0.178275,0.140370,0.175769,0.18592,0.104453
4,residual_deviance,50880.660000,10378.033000,56318.594000,40530.234000,44589.066000,46690.82400,66274.586000
5,rmse,224.663390,22.556100,237.315380,201.321230,211.161220,216.08060,257.438500
6,rmsle,0.516670,0.011745,NaN,0.508365,NaN,NaN,0.524975



Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance
0,,2022-04-24 08:31:40,0.976 sec,0.0,245.415475,94.108719,60228.755146
1,,2022-04-24 08:31:40,1.002 sec,5.0,233.823938,79.287539,54673.634105
2,,2022-04-24 08:31:40,1.026 sec,10.0,225.718956,72.325446,50949.046899
3,,2022-04-24 08:31:40,1.050 sec,15.0,221.017300,69.042994,48848.646865
4,,2022-04-24 08:31:40,1.075 sec,20.0,216.409429,67.425900,46833.041173
5,,2022-04-24 08:31:40,1.099 sec,25.0,213.251722,66.203481,45476.297024
6,,2022-04-24 08:31:40,1.126 sec,30.0,210.216463,65.295358,44190.961338
7,,2022-04-24 08:31:40,1.159 sec,35.0,207.192747,64.892540,42928.834268
8,,2022-04-24 08:31:40,1.183 sec,40.0,205.675019,64.492306,42302.213510
9,,2022-04-24 08:31:40,1.206 sec,45.0,203.658558,64.265632,41476.808353



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,room_type,514417888.0,1.000000,0.206614
1,neighbourhood,458230432.0,0.890775,0.184047
2,availability_365,427792480.0,0.831605,0.171821
3,calculated_host_listings_count,328708992.0,0.638992,0.132025
4,minimum_nights,262120096.0,0.509547,0.105280
5,reviews_per_month,183050608.0,0.355840,0.073522
6,neighbourhood_group,168548848.0,0.327650,0.067697
7,number_of_reviews,146880480.0,0.285528,0.058994


In [73]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)

model_id,rmse,mse,mae,rmsle,mean_residual_deviance
GBM_3_AutoML_2_20220424_83130,225.406,50807.7,69.3814,nan,50807.7
StackedEnsemble_BestOfFamily_2_AutoML_2_20220424_83130,225.425,50816.4,68.7847,nan,50816.4
StackedEnsemble_BestOfFamily_3_AutoML_2_20220424_83130,225.628,50908,68.6331,nan,50908
StackedEnsemble_AllModels_1_AutoML_2_20220424_83130,225.71,50945,68.5467,nan,50945
StackedEnsemble_AllModels_3_AutoML_2_20220424_83130,225.898,51029.9,68.9316,nan,51029.9
GBM_2_AutoML_2_20220424_83130,226.318,51219.7,69.7875,nan,51219.7
StackedEnsemble_AllModels_2_AutoML_2_20220424_83130,226.338,51229,68.6263,nan,51229
GBM_grid_1_AutoML_2_20220424_83130_model_8,226.64,51365.7,69.8674,nan,51365.7
GBM_grid_1_AutoML_2_20220424_83130_model_15,226.663,51376.1,70.1982,nan,51376.1
GBM_4_AutoML_2_20220424_83130,226.838,51455.3,70.6838,nan,51455.3


In [79]:
# To generate predictions on a test set, you can make predictions
# directly on the `H2OAutoML` object or on the leader model
# object directly
preds = aml.predict(datasetAutoMLTest)

gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


In [80]:
# Get leaderboard with all possible columns
lb = h2o.automl.get_leaderboard(aml, extra_columns = "ALL")
lb

model_id,rmse,mse,mae,rmsle,mean_residual_deviance,training_time_ms,predict_time_per_row_ms,algo
GBM_3_AutoML_2_20220424_83130,225.406,50807.7,69.3814,nan,50807.7,255,0.009766,GBM
StackedEnsemble_BestOfFamily_2_AutoML_2_20220424_83130,225.425,50816.4,68.7847,nan,50816.4,228,0.014368,StackedEnsemble
StackedEnsemble_BestOfFamily_3_AutoML_2_20220424_83130,225.628,50908,68.6331,nan,50908,244,0.014333,StackedEnsemble
StackedEnsemble_AllModels_1_AutoML_2_20220424_83130,225.71,50945,68.5467,nan,50945,235,0.016149,StackedEnsemble
StackedEnsemble_AllModels_3_AutoML_2_20220424_83130,225.898,51029.9,68.9316,nan,51029.9,320,0.024008,StackedEnsemble
GBM_2_AutoML_2_20220424_83130,226.318,51219.7,69.7875,nan,51219.7,251,0.006111,GBM
StackedEnsemble_AllModels_2_AutoML_2_20220424_83130,226.338,51229,68.6263,nan,51229,249,0.013915,StackedEnsemble
GBM_grid_1_AutoML_2_20220424_83130_model_8,226.64,51365.7,69.8674,nan,51365.7,187,0.005045,GBM
GBM_grid_1_AutoML_2_20220424_83130_model_15,226.663,51376.1,70.1982,nan,51376.1,317,0.007542,GBM
GBM_4_AutoML_2_20220424_83130,226.838,51455.3,70.6838,nan,51455.3,312,0.007594,GBM


## Shap Analysis

In [1]:
!pip3 install shap

In [46]:
import shap

In [62]:
# shap_values = shap.LinearExplainer(OLS_model).shap_values(X_train)
explainer = shap.Explainer(OLS_model,X_train, feature_dependence = "independent")

Exception: The passed model is not callable and cannot be analyzed directly with the given masker! Model: <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x7fddb8e57160>

In [55]:
shap.summary_plot(shap_values, X_train, plot_type="bar")

NameError: name 'shap_values' is not defined

In [63]:
exp = explainer.explain_instance(X_test[i], lin_reg.predict, num_features=5)

NameError: name 'explainer' is not defined